In [10]:
pip install pygame


In [6]:
import pygame
import time
from queue import Queue
from queue import PriorityQueue

class Node:
    def __init__(self, state, parent=None, cost=0, collected_food=None):
        self.state = state
        self.parent = parent
        self.path_cost = cost
        self.collected_food = collected_food if collected_food else set()

    def __lt__(self, node):
        return self.path_cost < node.path_cost
        
class Problem:
    def __init__(self, maze, start, goals):
        self.maze = maze[:]
        self.start = start
        self.goals = set(goals)  # for O(1) in lookup

    def test_goal(self, node):
        return len(node.collected_food) == len(self.goals)

    def actions(self, node):
        row, col = node.state
        moves = []
        if row > 0 and self.maze[row - 1][col] != '#':  # Up
            moves.append(1)
        if row < len(self.maze) - 1 and self.maze[row + 1][col] != '#':  # Down
            moves.append(3)
        if col < len(self.maze[0]) - 1 and self.maze[row][col + 1] != '#':  # Right
            moves.append(2)
        if col > 0 and self.maze[row][col - 1] != '#':  # Left
            moves.append(4)
        return moves

    def cost(self):
        return 1
def generate_child(problem, node, action):
    row, col = node.state
    if action == 1:
        new_state = (row-1, col)
    elif action == 2:
        new_state = (row, col+1)
    elif action == 3:
        new_state = (row+1, col)
    elif action == 4:
        new_state = (row, col-1)

    collected_food = set(node.collected_food)
    if new_state in problem.goals and new_state not in collected_food:
        collected_food.add(new_state)

    return Node(new_state, node, node.path_cost + problem.cost(), collected_food)
    
def generate_heuristics(node, problem):
    heuristics = float('inf')
    pacman_x, pacman_y = node.state
    food_locations = problem.goals - node.collected_food
    for food in food_locations:
        food_x, food_y = food
        distance = abs(pacman_x - food_x) + abs(pacman_y - food_y)
        if distance < heuristics:
            heuristics = distance
    return heuristics
    
def reconstruct_path(node):
    path = []
    while node:
        path.append(node.state)
        node = node.parent
    return path[::-1]  # Reverse to get correct order

def A_star(problem, heuristic):
    def reconstruct_path(node):
        path = []
        while node:
            path.append(node.state)
            node = node.parent
        return list(reversed(path))

    start_node = Node(problem.start)
    frontier = PriorityQueue()
    frontier.put((0, start_node))  # Priority queue: (f(n), node)
    explored = set()

    while not frontier.empty():
        _, current_node = frontier.get()

        if problem.test_goal(current_node):
            return reconstruct_path(current_node)

        explored_key = (current_node.state, frozenset(current_node.collected_food))
        if explored_key in explored:
            continue
        explored.add(explored_key)


        for action in problem.actions(current_node):
            child = generate_child(problem, current_node, action)
            child_key = (child.state, frozenset(child.collected_food))

            if child_key not in explored:
                # Calculate f(n) = g(n) + h(n)
                g_n = child.path_cost  # Actual cost from start to current node
                h_n = heuristic(child, problem )  # Heuristic estimate
                f_n = g_n + h_n

                # Check if the child is already in the frontier with a higher cost
                in_frontier = False
                for i, (priority, node) in enumerate(frontier.queue):
                    if node.state == child.state and frozenset(node.collected_food) == frozenset(child.collected_food):
                        in_frontier = True
                        if priority > f_n:
                            # Replace the node in the frontier with the better one
                            frontier.queue[i] = (f_n, child)
                        break

                if not in_frontier:
                    frontier.put((f_n, child))

    return None  # No solution found
        
def video_game(problem):
    WHITE = (0, 0, 0)
    BLACK = (0,0,88)
    YELLOW = (255, 255, 255)
    GREEN = (0, 0, 0)
    RED = (255,255,26)
    CELL_SIZE = 35
    ROWS = 4
    COLS = 10
    def draw_maze(screen, maze, path, collected_food):
        for row in range(ROWS):
            for col in range(COLS):
                rect = pygame.Rect(col * CELL_SIZE, row * CELL_SIZE, CELL_SIZE, CELL_SIZE)
                if maze[row][col] == '#':
                    pygame.draw.rect(screen, BLACK, rect)
                elif (row, col) in collected_food:
                    pygame.draw.rect(screen, GREEN, rect)
                elif maze[row][col] == 'o':
                    pygame.draw.circle(screen, YELLOW, rect.center, CELL_SIZE // 4)
            
    pygame.init()
    screen = pygame.display.set_mode((COLS * CELL_SIZE, ROWS * CELL_SIZE))
    pygame.display.set_caption("Pacman BFS")
    maze = problem.maze
    s = problem.start
    g = problem.goals

    path = A_star(problem,generate_heuristics)
    collected_food = set()
    clock = pygame.time.Clock()
    
    for pos in path:
        screen.fill(WHITE)
        draw_maze(screen, maze, path, collected_food)
        pygame.draw.circle(screen, RED, (pos[1] * CELL_SIZE + CELL_SIZE // 2, pos[0] * CELL_SIZE + CELL_SIZE // 2), CELL_SIZE // 3)
        collected_food.add(pos) if pos in g else None
        pygame.display.flip()
        clock.tick(3)
        
    time.sleep(2)
    pygame.quit()
    
def main():
    maze = [
        ['#', 'o', 'o', '.', '.', '.', '.', '.', 'o', 'o'],
        ['o', 'o', '#', '#', '#', '#', 'o', '.', 'o', 'o'],
        ['.', 'o', '#', '#', '#', '#', '#', '.', 'o', '#'],
        ['#', 'o', 'o', 'o', '#', '#', '#', '#', 'o', '.'],
    ]
    s = (2, 0)
    g = {(0, 1), (0, 2), (0, 8), (0, 9), (1, 0), (1, 1), (1, 6), (1, 8), (1, 9),(2,1), (2, 8), (3, 1), (3, 2), (3, 3), (3, 8)}
    prob = Problem(maze,s,g)
    video_game(prob)
    
main()



In [11]:
import itertools

class Node:
    def __init__(self, state, parent=None):
        self.state = state  # Represents the route as a list of locations
        self.parent = parent
        self.h = 0  # Heuristic value (total distance of the route)

    def __lt__(self, node):
        return self.h < node.h


class Problem:
    def __init__(self, start, goals, graph):
        self.start = start  # Initial route (e.g., ['d1', 'c1', 'c2', 'c3', 'd1'])
        self.goals = goals  # List of goals to visit
        self.graph = graph  # Graph representation of distances

    def evaluate(self, node):
        """Calculate the total distance of the route."""
        h = 0
        for i in range(len(node.state) - 1):
            node1 = node.state[i]
            node2 = node.state[i + 1]
            if node1 in self.graph and node2 in self.graph[node1]:
                distance = self.graph[node1][node2]
                h += distance
            else:
                h += float('inf')  # Penalize invalid routes
        node.h = h





def HC(problem):
    """Hill Climbing algorithm to solve the delivery problem."""
    current = Node(problem.start)
    problem.evaluate(current)

    while True:
        neighbor = height_valued_neighbor(current, problem)
        if neighbor.h >= current.h:  # No better neighbor found
            return current
        current = neighbor  # Move to the best neighbor


def main():
    # Define the graph (distances between locations)
    graph = {
        'd1': {'c1': 10, 'c2': 15, 'c3': 20, 'c4': 19},
        'c1': {'d1': 10, 'c2': 5, 'c3': 12, 'c4': 9},
        'c2': {'d1': 15, 'c1': 5, 'c3': 8, 'c4': 4},
        'c3': {'d1': 20, 'c1': 12, 'c2': 8, 'c4': 39},
        'c4': {'d1': 19, 'c1': 9, 'c2': 4, 'c3': 39}
    }

    # Define the initial goals and route
    goals = ['c1', 'c2', 'c3', 'c4']
    start_route = ['d1', 'd1']  # Start and end at the depot

    # Create the problem instance
    prob = Problem(start_route, goals, graph)

    # Run Hill Climbing
    result = HC(prob)
    print("Best Route:", result.state)
    print("Total Distance:", result.h)


if __name__ == "__main__":
    main()


[(['d1', 'c2', 'c1', 'c3', 'd1'], ['c1', 'c2', 'c3']), (['d1', 'c3', 'c2', 'c1', 'd1'], ['c1', 'c2', 'c3']), (['d1', 'c1', 'c3', 'c2', 'd1'], ['c1', 'c2', 'c3'])]


In [ ]:
import random
import math

class Node:
    def __init__(self,state,act=None):
        self.state = state[:]
        self.action = act
        self.h = 0   
    def __lt__(self,node):
        return self.h < node.h  

class Problem:
    def __init__(self,initial):
        self.initial = initial
        self.goal = goal
    def h(self, node):
        h = 0
        w, l = node.state
        x, y = l
        # Check horizontal matches
        left = x
        while left - 1 >= 0 and self.initial[y][left - 1] == w:
            h += 1
            left -= 1
        right = x
        while right + 1 < len(self.initial[0]) and self.initial[y][right + 1] == w:
            h += 1
            right += 1
        # Check vertical matches
        up = y
        while up - 1 >= 0 and self.initial[up - 1][x] == w:
            h += 1
            up -= 1
        down = y
        while down + 1 < len(self.initial) and self.initial[down + 1][x] == w:
            h += 1
            down += 1
        node.h = h
        
    def actions(self, node):
        _, l = node.state
        x, y = l
        moves = []
        if x > 0 and self.initial[y][x - 1] != '#':  # swap left
            moves.append(1)
        if x < len(self.initial[0]) - 1 and self.initial[y][x + 1] != '#':  #swap Right
            moves.append(3)
        if y < len(self.initial) - 1 and self.initial[y + 1][x] != '#':  # swap down
            moves.append(2)
        if y > 0 and self.initial[y - 1][x] != '#':  # swap up
            moves.append(4)
        return moves

def generate_child(node,action):
    if(action == 1):
        st = [node.state[0], (node.state[1][0] - 1 , node.state[1][1])
    elif(action == 2 ):
        st = [node.state[0], (node.state[1][0] , node.state[1][1] + 1)
    elif(action == 3):
        st = [node.state[0], (node.state[1][0] + 1 , node.state[1][1])
    elif(action == 4):
        st = [node.state[0], (node.state[1][0] , node.state[1][1] - 1)
    return Node(st,action)
        
def HC(problem):
    def start(problem):
        """ find the best to start from it. """
    g = problem.goal[0]
    while(True):
        for i in range(len(problem.initial)):
            for j in range(len(problem.initial[0])):
                if problem.initial[i][j] == g :
                    return Node([g,(j,i)])
        return Node(problem.initial[0][0],(0,0)]
    def height_valued_neighbor(node, problem):
        """ Generate all possible neighbors by moving each queen up/down. """
        neighbors = []
        best_h = float('inf')
        for act in problem.actions(node):
            nody = generate_child(node,act)
            problem.h(nody)
            neighbors.append(nody)
        best_neighbor = max(neighbors, key=lambda n: n.h)
        return best_neighbor

    for _ in range(1000):
        neighbor = height_valued_neighbor(start(problem),problem) #highest-valued successor
    return current

def main():
    board = [
        ['B', 'Y', 'R', 'G'],
        ['Y', 'B', 'G', 'R'],
        ['R', 'G', 'B', 'Y'],
        ['G', '#', 'Y', 'B']
    ]
    goal = [('B',2)]
    prob = Problem(board,goal)

In [2]:
import random
import math
import pygame
import time

class Node:
    def __init__(self,state):
        self.state = state[:]
        self.h = 0   
    def __lt__(self,node):
        return self.h < node.h  
class Problem:
    def __init__(self,initial):
        self.initial = initial
    def h(self, node):
        h = 0
        subgrid_size = 3  
        size = len(node.state)
        for i in range(size):
            row_seen = set()
            col_seen = set()
            for j in range(size):
                # Row check
                if node.state[i][j] in row_seen:
                    h += 1
                elif node.state[i][j] != 0:
                    row_seen.add(node.state[i][j])
                
                # Column check
                if node.state[j][i] in col_seen:
                    h += 1
                elif node.state[j][i] != 0:
                    col_seen.add(node.state[j][i])

        # Subgrid Conflicts
        for box_x in range(0, size, subgrid_size):
            for box_y in range(0, size, subgrid_size):
                subgrid_seen = set()
                for i in range(subgrid_size):
                    for j in range(subgrid_size):
                        num = node.state[box_x + i][box_y + j]
                        if num in subgrid_seen:
                            h += 1
                        elif num != 0:
                            subgrid_seen.add(num)

        node.h = h
        return h


def GA(problem):
    def random_neighbor(problem):
        """ Generate random  neighbors by moving each queen up/down. """
        state = [row[:] for row in problem.initial]
        child = Node(state)
        for i in range(len(child.state)):
          for j in range(len(child.state[0])):
            if child.state[i][j] == 0:
              child.state[i][j] = random.randint(1,9)
        problem.h(child)
        return child
    def bird(x,y):
      c = random.randint(0, len(x)-1)  
      child = x[:c] + y[c:]  
      return child    
    def random_select(population):
        total_fitness = sum(1 / (1 + individual.h) for individual in population) 
        pick = random.uniform(0, total_fitness)
        current = 0
        for individual in population:
            current += 1 / (1 + individual.h)
            if current > pick:
                return individual
        return population[-1]    
    def mutate(child,problem):
        x = random.randint(1,9)
        while(True):
          i = random.randint(0,8)
          j = random.randint(0,8)
          if(problem.initial[i][j]==0):
            child.state[i][j]=x
            break
        problem.h(child)
        

    population = [random_neighbor(problem) for _ in range(1000)]

    for _ in range(1000):
        new_population = []
        for _ in range(len(population) // 2):
            x = random_select(population)
            y = random_select(population)
            child1 = Node(bird(x.state, y.state))
            problem.h(child1)
            child2 = Node(bird(y.state, x.state))
            problem.h(child2)
            if random.random() < 0.1:  
                mutate(child1,problem)
            if random.random() < 0.1:
                mutate(child2,problem)
            new_population.append(child1)
            new_population.append(child2)
        population = new_population
        population = sorted(population, key=lambda n: n.h)
        if(population[0].h==0):
            return population[0]
    return population[0]  

def video_game(problem,result):
    WHITE = (255, 255, 255)
    BLUE = (0, 0, 153)
    GREY = (64,64,64)
    CELL_SIZE = 35
    ROWS = 9
    COLS = 9
    WIDTH = COLS * CELL_SIZE
    HEIGHT = ROWS * CELL_SIZE
    
    def draw_grid(board):
        """Draws the Sudoku grid with the given board state."""
        screen.fill(WHITE)
        for row in range(ROWS):
            for col in range(COLS):
                x, y = col * CELL_SIZE, row * CELL_SIZE
                rect = pygame.Rect(x, y, CELL_SIZE, CELL_SIZE)
                pygame.draw.rect(screen, GREY, rect, 1)  # Draw cell borders
                
                num = board[row][col]
                if num != 0:  # If it's not an empty cell
                    text = font.render(str(num), True, BLUE)
                    screen.blit(text, (x + 15, y + 10))

        # Draw bold lines for 3x3 grids
        for i in range(0, WIDTH, CELL_SIZE * 3):
            pygame.draw.line(screen, BLACK, (i, 0), (i, HEIGHT), 3)
            pygame.draw.line(screen, BLACK, (0, i), (WIDTH, i), 3)
            
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Sudoku GA")
    font = pygame.font.Font(None, 36)  # Font for numbers
    
    draw_grid(problem.initial)
    pygame.display.flip()
    time.sleep(5)
    draw_grid(result)
    pygame.display.flip()

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        
    pygame.quit() 


def main():
    init = [
          [0, 6, 0, 2, 0, 0, 0, 7, 1],
          [4, 0, 5, 0, 0, 0, 0, 0, 2],
          [3, 0, 0, 0, 8, 0, 6, 9, 0],
          [2, 0, 0, 9, 0, 8, 7, 0, 0],
          [0, 9, 3, 0, 0, 0, 8, 0, 0],
          [0, 0, 6, 0, 0, 1, 0, 0, 9],
          [0, 8, 7, 0, 3, 0, 0, 0, 6],
          [6, 0, 0, 0, 0, 0, 5, 0, 7],
          [0, 0, 0, 0, 0, 9, 0, 2, 0]
    ]
    prob = Problem(init)
    result = GA(prob)
    video_game(prob,result)
main()

2 6 8 | 2 3 9 | 6 7 1 | 
4 4 5 | 9 6 2 | 5 2 2 | 
3 7 1 | 3 8 6 | 6 9 1 | 
2 9 4 | 9 4 8 | 7 1 7 | 
9 9 3 | 3 2 9 | 8 5 2 | 
8 8 6 | 9 8 1 | 9 4 9 | 
1 8 7 | 6 3 2 | 2 9 6 | 
6 2 6 | 4 7 3 | 5 1 7 | 
9 5 9 | 5 6 9 | 9 2 4 | 
Final heuristic: 70
